# 0. Preliminaries

In [1]:
import requests
import time
import pandas as pd
import json

In [2]:
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

#1. Find top AI Authors

In [ ]:
dataset = load_dataset("jamescalam/ai-arxiv")

In [ ]:
sorted(dataset['train']['published']) #2015 to 2023

In [ ]:
dataset_sorted = pd.DataFrame(dataset['train'])

In [ ]:
dataset_sorted = dataset_sorted.loc[dataset_sorted['published'].astype("int") > 20200000, :]

In [ ]:
def retrieve_first_author_list(list_in, position):
    if isinstance(list_in, list):
      index_pos = position-1 if position <= (len(list_in)-1) else (len(list_in)-1)
      first_author = list_in[index_pos]
    else:
      first_author = list_in
    return first_author

In [ ]:
dataset_sorted['first_author'] = dataset_sorted['authors'].apply(lambda x: retrieve_first_author_list(x, 1))
dataset_sorted['second_author'] = dataset_sorted['authors'].apply(lambda x: retrieve_first_author_list(x, 2))

In [ ]:
dataset_sorted_2023 = dataset_sorted[dataset_sorted['published'].astype("int") > 20230101]

In [ ]:
first_authors_2023 = pd.DataFrame(dataset_sorted_2023['first_author'].value_counts().reset_index())
first_authors_2023.columns = ['name', 'first_author_count_2023']
second_authors_2023 = pd.DataFrame(dataset_sorted_2023['second_author'].value_counts().reset_index())
second_authors_2023.columns = ['name', 'second_author_count_2023']
total_authors= pd.DataFrame(dataset_sorted['first_author'].value_counts().reset_index())
total_authors.columns = ['name', 'first_author_count']

total_authors_merged = total_authors.merge(first_authors_2023, on ="name", how = "left").merge(second_authors_2023, on = "name", how = "left").fillna(0)
total_authors_merged['total_2023_count'] = total_authors_merged['first_author_count_2023']+total_authors_merged['second_author_count_2023']

In [ ]:
top_four_authors = total_authors_merged.sort_values(by = "total_2023_count").tail(4)

In [ ]:
top_publications = dataset_sorted.loc[dataset_sorted['first_author'].isin(top_four_authors['name'].values), :]

In [ ]:
top_publications

,id,title,summary,source,authors,categories,comment,journal_ref,primary_category,published,updated,content,references,first_author,second_author
43,2209.07686,Text and Patterns: For Effective Chain of Thou...,The past decade has witnessed dramatic gains i...,http://arxiv.org/pdf/2209.07686,"[Aman Madaan, Amir Yazdanbakhsh]","[cs.CL, cs.AI, cs.LG]",Shortened version with additional results from...,None,cs.CL,20220916,20221013,"October 14, 2022\nTEXT AND PATTERNS : FOREFFEC...","[{'id': '2202.07206'}, {'id': '2109.03910'}, {...",Aman Madaan,Amir Yazdanbakhsh
74,2303.17651,Self-Refine: Iterative Refinement with Self-Fe...,"Like humans, large language models (LLMs) do n...",http://arxiv.org/pdf/2303.17651,"[Aman Madaan, Niket Tandon, Prakhar Gupta, Sky...","[cs.CL, cs.AI, cs.LG]","Code, data, and demo at https://selfrefine.info/",None,cs.CL,20230330,20230525,SELF-REFINE :\nIterative Refinement with Self-...,"[{'id': '2302.13971'}, {'id': '2204.14146'}, {...",Aman Madaan,Niket Tandon
289,2103.00453,Self-Diagnosis and Self-Debiasing: A Proposal ...,"When trained on large, unfiltered crawls from ...",http://arxiv.org/pdf/2103.00453,"[Timo Schick, Sahana Udupa, Hinrich Schütze]",[cs.CL],Accepted at TACL,None,cs.CL,20210228,20210909,Self-Diagnosis and Self-Debiasing:\nA Proposal...,"[{'id': '1909.05858'}, {'id': '2101.05783'}, {...",Timo Schick,Sahana Udupa
325,2302.07867,Learning Performance-Improving Code Edits,The waning of Moore's Law has shifted the focu...,http://arxiv.org/pdf/2302.07867,"[Aman Madaan, Alexander Shypula, Uri Alon, Mil...","[cs.SE, cs.AI, cs.LG, cs.PF]",Project website: https://pie4perf.com/. This v...,None,cs.SE,20230215,20230221,Learning Performance-Improving Code Edits\nLEA...,"[{'id': '1611.01989'}, {'id': '2204.05999'}, {...",Aman Madaan,Alexander Shypula
338,2302.13971,LLaMA: Open and Efficient Foundation Language ...,"We introduce LLaMA, a collection of foundation...",http://arxiv.org/pdf/2302.13971,"[Hugo Touvron, Thibaut Lavril, Gautier Izacard...",[cs.CL],None,None,cs.CL,20230227,20230227,LLaMA: Open and Efﬁcient Foundation Language M...,"[{'id': '2302.13971'}, {'id': '2212.12017'}, {...",Hugo Touvron,Thibaut Lavril
362,2208.11663,PEER: A Collaborative Language Model,Textual content is often the output of a colla...,http://arxiv.org/pdf/2208.11663,"[Timo Schick, Jane Dwivedi-Yu, Zhengbao Jiang,...",[cs.CL],None,None,cs.CL,20220824,20220824,PEER: A Collaborative Language Model\nTimo Sch...,"[{'id': '2201.11903'}, {'id': '2208.11663'}, {...",Timo Schick,Jane Dwivedi-Yu
379,2302.04761,Toolformer: Language Models Can Teach Themselv...,Language models (LMs) exhibit remarkable abili...,http://arxiv.org/pdf/2302.04761,"[Timo Schick, Jane Dwivedi-Yu, Roberto Dessì, ...",[cs.CL],None,None,cs.CL,20230209,20230209,Toolformer: Language Models Can Teach Themselv...,"[{'id': '1910.07475'}, {'id': '2110.14168'}, {...",Timo Schick,Jane Dwivedi-Yu
416,2305.03047,Principle-Driven Self-Alignment of Language Mo...,"Recent AI-assistant agents, such as ChatGPT, p...",http://arxiv.org/pdf/2305.03047,"[Zhiqing Sun, Yikang Shen, Qinhong Zhou, Hongx...","[cs.LG, cs.AI, cs.CL, cs.CY]",Project page: https://mitibmdemos.draco.res.ib...,None,cs.LG,20230504,20230504,Principle-Driven Self-Alignment of Language Mo...,"[{'id': '2212.10560'}, {'id': '2210.03629'}, {...",Zhiqing Sun,Yikang Shen
420,2307.09288,Llama 2: Open Foundation and Fine-Tuned Chat M...,"In this work, we develop and release Llama 2, ...",http://arxiv.org/pdf/2307.09288,"[Hugo Touvron, Louis Martin, Kevin Stone, Pete...","[cs.CL, cs.AI]",None,None,cs.CL,20230718,20230719,L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle : Open ...,"[{'id': '2302.13971'}, {'id': '1908.01091'}, {...",Hugo Touvron,Louis Martin


# 2. Load Author Publication Data

In [39]:
sample_papers = pd.read_csv("../data/Authors_Dataset_complete_UTF8.csv")

In [40]:
sample_papers.head()

,Author,Title,Year,Link,Abstract,Introduction,Conclusion
0,Aman Madaan,AutoMix: Automatically Mixing Language Models,2023,https://arxiv.org/pdf/2310.12963.pdf,Large language models (LLMs) are now available...,Human problem-solving inherently follows a mul...,AutoMix integrates black-box large language mo...
1,Aman Madaan,Self-refine: Iterative refinement with self-fe...,2023,https://arxiv.org/pdf/2303.17651,"Like humans, large language models (LLMs) do n...",Although large language models (LLMs) can gene...,We present SELF-REFINE: a novel approach that ...
2,Aman Madaan,Learning performance-improving code edits,2023,https://arxiv.org/pdf/2302.07867,The waning of Moore’s Law has shifted the focu...,Ensuring efficiency is a crucial aspect of pro...,This work explores the capability of language ...
3,Aman Madaan,Language models of code are few-shot commonsen...,2022,https://arxiv.org/pdf/2210.07128,We address the general task of structured comm...,The growing capabilities of large pre-trained ...,We present the first work to employ large lang...
4,Aman Madaan,Text and patterns: For effective chain of thou...,2022,https://arxiv.org/pdf/2209.07686,"In the past decade, we witnessed dramatic gain...",The ability to learn a previously unseen task ...,This work evaluates the capacity of COT to ele...


In [41]:
sample_papers[sample_papers['Introduction'].isnull()]

,Author,Title,Year,Link,Abstract,Introduction,Conclusion
16,Aman Madaan,Analysis of a Convex Formulation for Distant S...,2015,https://madaan.github.io/res/papers/analysis-c...,TO DELETE,NaN,NaN
17,Aman Madaan,"Occurrence Statistics of Entities, Relations a...",2014,NaN,TO DELETE,NaN,NaN
63,Timo Schick,Few-shot learning with language models: Learni...,2022,this is a PHD ziziz,NaN,NaN,NaN


In [42]:
sample_papers_auth = sample_papers[['Author', 'Abstract', 'Introduction', 'Conclusion']].dropna(subset = ['Abstract', 'Introduction', 'Conclusion']).reset_index(drop = True)

In [43]:
sample_papers_auth['Author'].value_counts()

Author
Aman Madaan     16
Hugo Touvron    15
Zhiqing Sun     15
Timo Schick     15
Name: count, dtype: int64

# 3. Generate AI Captions

Ideally, we should probably use the API to do this. But since we do not have it confirmed we can use ChatGPT for now.

In [44]:
def make_GPT_prompt(text):
  return f"{text} - can you write me an abstract, introduction and conclusion for the paper that is summarized above"

In [45]:
#TODO - maybe 16 papers is fine since it is the average now?
sample_papers_GPT = pd.DataFrame(columns=["Author", "Abstract"])
for author in sample_papers_auth["Author"].unique():
    random_pub = sample_papers_auth[sample_papers_auth["Author"] == author][['Author', 'Abstract']].sample(n=4, random_state = 42)
    sample_papers_GPT = pd.concat([sample_papers_GPT, random_pub], ignore_index = True)

sample_papers_GPT['GPT_prompt'] = sample_papers_GPT['Abstract'].apply(make_GPT_prompt)

In [46]:
sample_papers_GPT['Author'].value_counts()

Author
Aman Madaan     4
Hugo Touvron    4
Zhiqing Sun     4
Timo Schick     4
Name: count, dtype: int64

In [119]:
sample_papers_GPT_more= pd.DataFrame(columns=["Author", "Abstract"])
for author in sample_papers_auth["Author"].unique():
    random_pub = sample_papers_auth[sample_papers_auth["Author"] == author][['Author', 'Abstract']].sample(n=6, random_state = 42)
    sample_papers_GPT_more = pd.concat([sample_papers_GPT_more, random_pub], ignore_index = True)

sample_papers_GPT_more['GPT_prompt'] = sample_papers_GPT_more['Abstract'].apply(make_GPT_prompt)

In [121]:
sample_papers_GPT_more.to_csv("../data/GPT_data_UTF8_additional.csv", encoding = "utf-8")

In [120]:
sample_papers_GPT_more['Author'].value_counts()

Author
Aman Madaan     6
Hugo Touvron    6
Zhiqing Sun     6
Timo Schick     6
Name: count, dtype: int64

In [137]:
#filled out data
GPT_results = pd.read_csv("../data/GPT_data_UTF8_additional_2.csv", encoding = "utf-8")

In [139]:
GPT_results.head()

,Unnamed: 0,Author,Abstract,GPT_abstract,GPT_introduction,GPT_conclusion
0,0,Aman Madaan,Large language models (LLMs) are now available...,In the landscape of cloud-based Large Language...,The availability of diverse Large Language Mod...,AutoMix emerges as a promising solution in the...
1,1,Aman Madaan,"Like humans, large language models (LLMs) do n...",Large Language Models (LLMs) often fall short ...,Large Language Models (LLMs) have exhibited re...,"In this study, we introduced SELF-REFINE, a no..."
2,2,Aman Madaan,Machine learning systems typically apply the\n...,Machine learning systems often lack the adapti...,Machine learning models commonly lack the nuan...,"In this work, we introduced FLOWGEN, a graph g..."
3,3,Aman Madaan,We propose a method of curating high-quality c...,"In this paper, we present an innovative method...",Training data scarcity remains a formidable ch...,"In this work, we introduced a method for curat..."
4,4,Aman Madaan,This paper introduces a new task of politeness...,This paper introduces a novel task in the doma...,"In the realm of natural language processing, t...","In conclusion, this paper introduces and addre..."


In [140]:
GPT_results['Author'] = "GPT-3.5"

# 4. Join Datasets and Chunk it

In [141]:
#TODO - to replace with filled out data
GPT_results.columns = ["ind", "Author", "Prompt", "Abstract", "Introduction", "Conclusion"]
GPT_results['Author'] = "GPT-3.5"
joined_data = pd.concat([GPT_results[['Author', 'Abstract', 'Introduction', 'Conclusion']], sample_papers_auth[['Author', 'Abstract', 'Introduction', 'Conclusion']]])

In [142]:
def expand_dataframe(df, col_Name):
    rows = []
    for index, row in df.iterrows():
        name = row['Author']
        name_list = row[col_Name]
        for item in name_list:
            rows.append([name, name_list, col_Name, item])

    new_df = pd.DataFrame(rows, columns=['Author', 'Pub', 'Type', 'Chunk'])
    return new_df

In [66]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [67]:
tokenizer_reg = nltk.RegexpTokenizer(r"\w+")

In [143]:
tok_ratio = 3/4
max_seq =300

def split_text_to_chunks(text):
  list_sequences = []
  list_sentences = nltk.tokenize.sent_tokenize(text)
  length_word_sentences = [len(tokenizer_reg.tokenize(sen)) for sen in list_sentences]
  sum_tokens = sum(length_word_sentences)
  #parse through each sentence
  sequence_out = ""
  total_token_count = 0
  token_count = 0
  min_chunk_length = 100
  for sentence in list_sentences:
      words = tokenizer_reg.tokenize(sentence)
      if (len(words)+token_count < int(max_seq*tok_ratio)) or ((sum_tokens-total_token_count-len(words)) < min_chunk_length):
        sequence_out = sequence_out+" "+sentence
        token_count = token_count + len(words)
      else:
        list_sequences.append(sequence_out)
        sequence_out = sentence
        token_count = len(words)
      total_token_count += len(words)

  list_sequences.append(sequence_out)
  return list_sequences




In [144]:
joined_data['abstract_chunked'] = joined_data['Abstract'].apply(split_text_to_chunks)
abstract_chunked = expand_dataframe(joined_data, 'abstract_chunked')

In [145]:
joined_data['intro_chunked'] = joined_data['Introduction'].apply(split_text_to_chunks)
intro_chunked = expand_dataframe(joined_data, 'intro_chunked')

In [146]:
joined_data['conclusion_chunked'] = joined_data['Conclusion'].apply(split_text_to_chunks)
conclusion_chunked = expand_dataframe(joined_data, 'conclusion_chunked')

In [147]:
joined_data = pd.concat([abstract_chunked, intro_chunked, conclusion_chunked])

In [148]:
joined_data

,Author,Pub,Type,Chunk
0,GPT-3.5,[ In the landscape of cloud-based Large Langua...,abstract_chunked,In the landscape of cloud-based Large Languag...
1,GPT-3.5,[ Large Language Models (LLMs) often fall shor...,abstract_chunked,Large Language Models (LLMs) often fall short...
2,GPT-3.5,[ Machine learning systems often lack the adap...,abstract_chunked,Machine learning systems often lack the adapt...
3,GPT-3.5,"[ In this paper, we present an innovative meth...",abstract_chunked,"In this paper, we present an innovative metho..."
4,GPT-3.5,[ This paper introduces a novel task in the do...,abstract_chunked,This paper introduces a novel task in the dom...
...,...,...,...,...
86,Timo Schick,[ We have devised a novel approach for the cha...,conclusion_chunked,We have devised a novel approach for the chal...
87,Timo Schick,[ We have devised a novel approach for the cha...,conclusion_chunked,As an exhaustive search for the highest-scorin...
88,Timo Schick,[ We have devised a novel approach for the cha...,conclusion_chunked,This result strongly suggests that our transit...
89,Timo Schick,[ We have devised a novel approach for the cha...,conclusion_chunked,"Of course, such a revision does not\nhave to b..."


In [149]:
joined_data[['Author', 'Chunk']].groupby("Author").count()

,Chunk
Author,
Aman Madaan,80
GPT-3.5,72
Hugo Touvron,74
Timo Schick,76
Zhiqing Sun,77


In [151]:
joined_data.to_csv("../data/chunked_author_data_UTF8.csv")

In [152]:
len(joined_data)

379

Irrelevant stuff from now on. to delete later

#5. Try out on a small sample

In [ ]:
# initialise the stop words
with open('stopwords.txt', 'r') as file:
    # Read each line, strip whitespace and store it in a list
    stopwords = [line.strip() for line in file]
    stopwords = set(stopwords)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import string
from sklearn.model_selection import train_test_split

punct_vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize, use_idf=False, norm='l1', vocabulary=string.punctuation)
stopword_bigram_vectorizer = CountVectorizer(ngram_range=(1, 3), tokenizer=nltk.word_tokenize, vocabulary=stopwords)

# Preparing the data
X = df_chunksplit['Chunk'].values
y = df_chunksplit['Author'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Feature Extractors
class LexicalDiversity(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, papers):
        return [[len(set(nltk.word_tokenize(paper))) / len(nltk.word_tokenize(paper))] for paper in papers]

class AverageWordsPerSentence(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, papers):
        return [[np.mean([len(sentence.split()) for sentence in nltk.sent_tokenize(paper)])] for paper in papers]

# Create feature extraction pipelines
lexical_pipeline = Pipeline([
    ('lexical_div', LexicalDiversity()),
    ('scaler', StandardScaler())
])

avg_words_pipeline = Pipeline([
    ('avg_words', AverageWordsPerSentence()),
    ('scaler', StandardScaler())
])

punct_vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize, use_idf=False, norm='l1', vocabulary=string.punctuation)

# For bigrams of stop words
stopword_bigram_vectorizer = CountVectorizer(ngram_range=(1, 3), tokenizer=nltk.word_tokenize, vocabulary=stopwords)

transformerss = [
    ('lexical_features', lexical_pipeline),
    ('avg_words_per_sentence', avg_words_pipeline),
    ('punctuation_features', punct_vectorizer),
    ('stopword_bigram_features', stopword_bigram_vectorizer)
]

# Combine all feature extractors
feature_pipeline = FeatureUnion(transformerss)

# Create a complete pipeline with feature extraction and SVM classifier
pipeline = Pipeline([
    ('features', feature_pipeline),
    ('classifier', SVC(kernel='linear'))
])

# Preparing the data
X = df_chunksplit['Chunk'].values
y = df_chunksplit['Author'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the pipeline
pipeline.fit(X_train, y_train)

# Extract feature names from each transformer
feature_names = []
for name, transformer in transformerss:
    feature_names.extend(transformer.transform(X_train))

# Predict
y_pred = pipeline.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [ ]:
feature_names

[array([-0.73246754]),
 array([0.45967121]),
 array([-0.62853862]),
 array([0.35623814]),
 array([0.82738211]),
 array([-0.67353182]),
 array([1.29069626]),
 array([-1.1301247]),
 array([0.72582529]),
 array([-0.26044347]),
 array([0.96533611]),
 array([1.4765237]),
 array([-1.72232867]),
 array([-0.48977285]),
 array([0.32113234]),
 array([-0.94334407]),
 array([-0.49727306]),
 array([0.37382266]),
 array([-0.45303977]),
 array([-1.15097062]),
 array([1.51695739]),
 array([0.40552809]),
 array([1.49483606]),
 array([0.68655666]),
 array([-0.38203853]),
 array([0.88556211]),
 array([0.38158726]),
 array([-1.14303366]),
 array([-0.31545272]),
 array([-1.25618625]),
 array([1.31449912]),
 array([0.13650871]),
 array([0.35623814]),
 array([0.25527872]),
 array([-1.2285394]),
 array([0.7122418]),
 array([1.5821571]),
 array([-0.41069311]),
 array([-0.39515982]),
 array([0.29659729]),
 array([0.09738638]),
 array([-0.44763619]),
 array([1.33682655]),
 array([0.24729588]),
 array([0.6343071]

In [ ]:
y_pred == y_test

array([False,  True, False, False, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True, False])

In [ ]:
!pip install eli5
import eli5
from eli5 import show_prediction, show_weights

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107719 sha256=648329f2aed4111fff22d7ab100cf856acac64a96db7d56bb492c3744635f785
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5


In [ ]:
show_weights(pipeline['classifier'])

In [ ]:
pipeline['features']['lexical_features']

TypeError: ignored